In [4]:
# # https://stackoverflow.com/questions/22994423/difference-between-np-random-seed-and-np-random-randomstate
# # np.random.seed(18)
# np.random.RandomState(18)

In [5]:
import os
import sys
import os.path as path

In [6]:
project_dir = '.'
data_dir = path.join('.','hvc_data')
url = path.join(data_dir, 'hvc_annotations.csv')

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [8]:
df = pd.read_csv(url)
df.drop('filename', axis=1, inplace = True)
df.head(2)

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg


In [9]:
df = df[:100]
df.shape

(100, 9)

In [10]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [11]:
path.abspath(path.join(data_dir,'processed'))

'D:\\Projects\\PersonAttributesData\\hvc_data\\processed'

In [12]:
os.getcwd()

'D:\\Projects\\PersonAttributesData'

In [13]:
# Custom batch generator
# -------
# Good one -  https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
# https://towardsdatascience.com/image-augmentation-14a0aafd0498
# https://towardsdatascience.com/writing-custom-keras-generators-fe815d992c5a    
# https://medium.com/the-artificial-impostor/custom-image-augmentation-with-keras-70595b01aeac
# https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

# Good one - https://www.kaggle.com/nikhilroxtomar/generators-for-keras-model


In [26]:
import keras
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator, img_to_array


# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]


class PersonDataGenerator(keras.utils.Sequence):
    """
    Ground truth data generator 

    https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence
    """
    
    def __init__(self, df, batch_size = 32, input_size = (224, 224), 
                 location = '.', augmentations = None, save_dir = '',
                 shuffle = False):
        self.df = df
        self.image_size = input_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentations #ImageDataGenerator instance
        self.location = location
        self.save_dir = path.abspath(save_dir) # path.abspath(path.join(self.location,'processed'))
        self.on_epoch_end()

        if not path.isdir(self.save_dir):
                os.mkdirs(self.save_dir, exist_ok=True)

    def __len__(self):
        """
        Number of batch in the Sequence.
        """
        return int(np.floor(self.df.shape[0] / self.batch_size))

    
    def __getitem__(self, index):
        """
        Gets batch at position index.
        fetch batched images and targets        
        """
        # slice function - https://www.w3schools.com/python/ref_func_slice.asp
            
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]

        images = np.stack([cv2.imread(path.join(self.location, item["image_path"])) for _, item in items.iterrows()])        
        if self.augmentation:
            if self.save_dir:
                images = self.augmentation.flow(images, 
                                            batch_size=self.batch_size, 
                                            save_to_dir=self.save_dir,
                                            save_prefix='aug').next()
                    
            else:
                images = self.augmentation.flow(images, 
                                            batch_size=self.batch_size).next()


        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        
        return images, target

    def on_epoch_end(self):
        """
        Shuffles/sample the df and thereby 
        updates indexes after each epoch
        
        Method called at the end of every epoch.
        
        """
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
            # frac --> take sample of the given df, sample size is given as fraction number
            # reset_index drop --> use the drop parameter to avoid the old index being added as a column
            # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
            

In [27]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=18)
train_df.shape, val_df.shape

((85, 28), (15, 28))

In [28]:
def blur(img):
    return (cv2.blur(img,(5,5)))

def get_random_eraser(input_img, p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    img_h, img_w, img_c = input_img.shape
    p_1 = np.random.rand()

    if p_1 > p:
        return input_img

    while True:
        s = np.random.uniform(s_l, s_h) * img_h * img_w
        r = np.random.uniform(r_1, r_2)
        w = int(np.sqrt(s / r))
        h = int(np.sqrt(s * r))
        left = np.random.randint(0, img_w)
        top = np.random.randint(0, img_h)

        if left + w <= img_w and top + h <= img_h:
            break

    if pixel_level:
        c = np.random.uniform(v_l, v_h, (h, w, img_c))
    else:
        c = np.random.uniform(v_l, v_h)

    input_img[top:top + h, left:left + w, :] = c

    return input_img
    

def blur_cutout(img):
    img =blur(img)
    img = get_random_eraser(img)
    return img

train_aug = ImageDataGenerator(rescale=1/255.0,
                                        horizontal_flip=True,
                                        rotation_range=30,
                                        brightness_range=[0.2,0.8],
                                        channel_shift_range=100,
                                        preprocessing_function=blur_cutout
                                    )

val_aug = ImageDataGenerator(rescale=1/255.0)

In [29]:
# create train and validation data generators


train_gen = PersonDataGenerator(train_df, batch_size=32, 
                                input_size=(224, 224), location = data_dir, 
                                augmentations=train_aug, shuffle=True)

valid_gen = PersonDataGenerator(train_df, batch_size=32, 
                                input_size=(224, 224), location = data_dir, 
                                augmentations=val_aug)

In [30]:
images, targets = next(iter(train_gen))

In [31]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'gender': 2,
 'image_quality': 3,
 'age': 5,
 'weight': 4,
 'bag': 3,
 'pose': 3,
 'footwear': 3,
 'emotion': 4}

In [20]:
# %tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

# from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [21]:
backbone = VGG16(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    # redundant
#     neck = Dropout(0.2)(in_layer)
#     neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, incoming):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(incoming)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [22]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = False

In [23]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [35]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    epochs=5)
#     use_multiprocessing=True,
#     workers=6, 
#     epochs=10,


Epoch 1/5


MemoryError: 

In [436]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [437]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2785027350564102718
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4951913267
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4807772686025660392
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [438]:
import tensorflow as tf
tf.test.is_gpu_available() # True/False

True

In [439]:
# Or only check for gpu's with cuda support
tf.test.is_gpu_available(cuda_only=True) 

True

In [234]:
a = np.random.uniform(low=0, high=255, size=(10,32,32,3))

In [235]:
a

array([[[[1.27585603e+02, 3.25460126e+01, 7.79283066e+01],
         [1.70839082e+02, 1.57297408e-01, 3.86185582e+01],
         [1.76328884e+02, 1.54596652e+02, 2.33121628e+02],
         ...,
         [2.31363788e+02, 5.71377869e+00, 7.16094464e+01],
         [2.23426236e+02, 1.55311589e+01, 1.24620324e+02],
         [1.54591580e+02, 2.16731602e+02, 2.15383715e+02]],

        [[2.44851704e+02, 3.24262800e+01, 6.03719302e+01],
         [1.03985182e+02, 1.64772514e+02, 1.79673482e+02],
         [2.17943032e+02, 1.25824198e+02, 5.00939881e+00],
         ...,
         [2.11637380e+02, 1.47452629e+02, 2.07640810e+02],
         [4.95952850e+01, 1.62173611e+02, 4.79476001e+01],
         [1.75768877e+02, 3.95225851e+01, 2.16237439e+02]],

        [[6.06966513e+01, 2.47324812e+02, 1.92106703e+02],
         [4.36843349e+01, 2.46826788e+02, 2.17501069e+02],
         [1.99176266e+02, 2.10746780e+02, 1.39805801e+02],
         ...,
         [6.01348524e+01, 4.99465160e+01, 1.65431186e+02],
         [

In [236]:
a = a/255.0

In [237]:
a

array([[[[5.00335700e-01, 1.27631422e-01, 3.05601202e-01],
         [6.69957183e-01, 6.16852580e-04, 1.51445326e-01],
         [6.91485818e-01, 6.06261382e-01, 9.14202461e-01],
         ...,
         [9.07308973e-01, 2.24069753e-02, 2.80821358e-01],
         [8.76181317e-01, 6.09065054e-02, 4.88707155e-01],
         [6.06241489e-01, 8.49927852e-01, 8.44642019e-01]],

        [[9.60202760e-01, 1.27161883e-01, 2.36752668e-01],
         [4.07785029e-01, 6.46166722e-01, 7.04601892e-01],
         [8.54678555e-01, 4.93428226e-01, 1.96447012e-02],
         ...,
         [8.29950509e-01, 5.78245604e-01, 8.14277684e-01],
         [1.94491314e-01, 6.35974947e-01, 1.88029804e-01],
         [6.89289715e-01, 1.54990530e-01, 8.47989958e-01]],

        [[2.38026084e-01, 9.69901223e-01, 7.53359618e-01],
         [1.71311117e-01, 9.67948188e-01, 8.52945369e-01],
         [7.81083396e-01, 8.26457959e-01, 5.48258045e-01],
         ...,
         [2.35822951e-01, 1.95868690e-01, 6.48749749e-01],
         [